In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [2]:
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [3]:
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from os import listdir
from os.path import join
from torchvision import transforms

class SIGNSDataset(Dataset):
    def __init__(self, data_dir):
        # Store the paths to the image files and their corresponding labels
        labels = listdir(data_dir)
        self.filenames = []
        self.labels = []

        for i, label in enumerate(labels):
            label_dir = join(data_dir, label)
            for filename in listdir(label_dir):
                file_path = join(label_dir, filename)
                self.filenames.append(file_path)
                self.labels.append(i)
        
        self.transform = transforms.Compose([
            transforms.Resize(64),              
            transforms.RandomHorizontalFlip(),  
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])
        ]) 

    def __len__(self):
        # Return the size of the dataset
        return len(self.filenames)
    
    def __getitem__(self, idx):
        img_path = self.filenames[idx]
        image = Image.open(img_path)
        image = self.transform(image)
        label = self.labels[idx]
        return image, label

In [4]:
train_dataset = SIGNSDataset('Train_Dataset')

In [5]:
batch_size = 64

In [6]:
train_loader = DataLoader(dataset = train_dataset,batch_size = batch_size,shuffle = True)

In [7]:
test_dataset = SIGNSDataset('Test_Dataset')

In [8]:
test_loader = DataLoader(dataset = test_dataset,batch_size = batch_size,shuffle = True)

In [9]:
class NN(nn.Module):
    def __init__(self,input_size,num_classes):
        super(NN,self).__init__()
        self.layer1 = nn.Linear(input_size,64)
        self.layer2 = nn.Linear(64,num_classes)

    def forward(self,x):
        x = self.layer1(x)
        x = torch.relu(x)
        x = self.layer2(x)
        output = torch.softmax(x,dim=1)
        return output

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
input_size = 12288
num_classes = 10
learning_rate = 0.001
num_epochs = 50

In [25]:
model = NN(input_size = input_size,num_classes = num_classes).to(device)

In [26]:
criterion = nn.CrossEntropyLoss()

In [27]:
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

In [45]:
for epoch in range(num_epochs):
    for batch_idx, (data,target) in enumerate(train_loader):
        data = data.to(device = device)
        targets = target.to(device = device)
        data = data.reshape(data.shape[0],-1)
        scores = model(data)
        loss = criterion(scores,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch == num_epochs-1:
        PATH = './Sign_net_' + str(epoch) + '.pth'
        torch.save(model.state_dict(),PATH)
        

In [46]:
def check_accuracy(loader,model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x,y in loader:
            x = x.to(device = device)
            y = y.to(device = device)
            x = x.reshape(x.shape[0],-1)
            model.load_state_dict(torch.load(f'Sign_net_{num_epochs-1}.pth', weights_only=True))
            model.to(device = device)
            scores = model(x)
            _,predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    return num_correct,num_samples

In [47]:
num_correct,num_samples = check_accuracy(test_loader,model)
print(int(num_correct),"out of",num_samples)

171 out of 249
